In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from utils import generate_data
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import time

In [2]:
np.random.seed(1)
data = generate_data()

In [3]:
gpois_mdl = sm.GeneralizedPoisson(data.y, data.x_df)
gpois_res = gpois_mdl.fit()
print(gpois_res.summary2())

Optimization terminated successfully.
         Current function value: 6.042689
         Iterations: 20
         Function evaluations: 23
         Gradient evaluations: 23
                       Results: GeneralizedPoisson
Model:                 GeneralizedPoisson  Pseudo R-squared:  0.046      
Dependent Variable:    y                   AIC:               13257.4897 
Date:                  2022-04-02 14:34    BIC:               13317.4718 
No. Observations:      1095                Log-Likelihood:    -6616.7    
Df Model:              10                  LL-Null:           -6938.4    
Df Residuals:          1084                LLR p-value:       9.1750e-132
Converged:             1.0000              Scale:             1.0000     
-------------------------------------------------------------------------
                          Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
-------------------------------------------------------------------------
intercept                 5.2691   0.

## Extend Generic Likelihood Model

In [4]:
def _vec_matrix_multiply(a, B):
    return np.apply_along_axis(lambda x: x * a, 0, B)

def _ll_latentnorm(y, X, beta, alph):
    mu = (np.dot(X, beta)) # Should we exponentiate this??
    sigma = np.exp(np.dot(X, alph))
    Phi_bar = stats.norm(mu, sigma).cdf(np.log1p(y))
    Phi_underbar = stats.norm(mu, sigma).cdf(np.log(y))
    ll = np.log(Phi_bar - Phi_underbar)
    #print(Phi_bar.sum())
    #print(Phi_underbar.sum())
    #print(ll.sum())
    return ll

def _gradutils(y, X, beta, alph):
    mu = (np.dot(X, beta)) # Should we exponentiate this??
    sigma = np.exp(np.dot(X, alph))

    z_bar = (np.log1p(y) - mu) / sigma 
    z_underbar = (np.log(y) - mu) / sigma

    Phi_bar = stats.norm.cdf(z_bar)
    Phi_underbar = stats.norm.cdf(z_underbar)
    Phi  = Phi_bar - Phi_underbar

    phi_bar = stats.norm.pdf(z_bar)
    phi_underbar = stats.norm.pdf(z_underbar)
    phi = phi_bar - phi_underbar
    
    kappa_0 = phi / Phi
    kappa_1 = (z_bar * phi_bar - z_underbar * phi_underbar) / Phi
    kappa_2 = (z_bar**2 * phi_bar - z_underbar**2 * phi_underbar) / Phi
    kappa_3 = (z_bar**3 * phi_bar - z_underbar**3 * phi_underbar) / Phi
    
    return kappa_0, kappa_1, kappa_2, kappa_3, mu, sigma

def _em_gradutils(W, sigma, c, alpha, return_hessian=False):
    sigma_neg_2 = sigma**-2
    grad = W.T @ (sigma_neg_2 * c - 1.) - alpha
    hessian = None
    if return_hessian:
        W_sqrt_k = _vec_matrix_multiply(np.sqrt(c)/sigma, W)
        hessian = -2. * W_sqrt_k.T @ W_sqrt_k
    return grad, hessian

In [5]:
class MyLatentNormalEM(GenericLikelihoodModel):
    def __init__(self, endog, exog, penalty=0., **kwds):
        super(MyLatentNormalEM, self).__init__(endog, exog, **kwds)
        self.nparams = 22
        self.penalty = penalty
        
        
    def nloglikeobs(self, params):
        exog = self.exog
        endog = self.endog
        beta = params[:11] #first 11 are for mu
        alph = params[11:] #last 11 are for sigma
        ll = _ll_latentnorm(endog, exog, beta, alph)
        params_alt = params.copy()
        params_alt[0] = 0.
        return -ll + self.penalty*np.sum(params_alt**2)/self.endog.size
    
    def score(self, params):
        X = self.exog
        y = self.endog
        beta = params[:11] #first 11 are for mu
        alph = params[11:] #last 11 are for sigma
        
        kappa_0, kappa_1, kappa_2, kappa_3, mu, sigma = _gradutils(y, X, beta, alph)
        
        beta_alt = beta.copy()
        beta_alt[0] = 0
        alph_alt = alph.copy()
#         alph_alt[0] = 0.
        
        grad_beta = -(kappa_0 / sigma) @ X - self.penalty*2 * beta_alt
        grad_alph = -kappa_1 @ X - self.penalty*2 * alph_alt
        
        return np.append(grad_beta, grad_alph)
    
    def hessian(self, params):
        y = self.endog
        X = self.exog
        beta = params[:11] #first 11 are for mu
        alph = params[11:] #last 11 are for sigma
        
        kappa_0, kappa_1, kappa_2, kappa_3, mu, sigma = _gradutils(y, X, beta, alph)
        
        k_beta = (kappa_0**2 + kappa_1) / sigma**2
        k_alph = kappa_1 * (kappa_1 - 1) + kappa_3
        k_beta_alph = (kappa_2 + kappa_0*(kappa_1 - 1)) / sigma
        H_beta = np.zeros([11, 11])
        H_alph = np.zeros([11, 11])
        H_beta_alph = np.zeros([11, 11])
                  
        for i in range(X.shape[0]):
            x = X[i]
            xxT = np.outer(x, x)
            H_beta -= k_beta[i] * xxT
            H_alph -= k_alph[i] * xxT
            H_beta_alph -= k_beta_alph[i] * xxT
        
        H_all = np.block([[H_beta, H_beta_alph], [H_beta_alph.T, H_alph]]) # 22 x 22
        penalty_matrix = self.penalty*2 * np.eye(22)
        penalty_matrix[0, 0] = 0.

        return H_all - penalty_matrix
        
    
    def fit(self, start_params=None, maxiter=100, use_hessian=False, step_size=1e-4, tol=1e-6):
        start_time = time.time()
        if start_params is None:
            # Reasonable starting values
            start_params = np.zeros(self.nparams)
            start_params[0] = np.log(np.mean(self.endog)) # beta intercept
        self.em(
            start_params=start_params,
            maxiter=maxiter,
            use_hessian=use_hessian,
            step_size=step_size,
            tol=tol)
        end_time = time.time()
        self.wall_time = end_time - start_time
        return self
    
    
    def update_beta(self, beta):
        self.beta = beta
        self.mu = self.exog @ beta
        
        
    def update_alpha(self, alpha):
        self.alpha = alpha
        self.sigma = np.exp(self.exog @ alpha)
    
    
    def update_expectations(self):
        kappa_0, kappa_1, kappa_2, kappa_3, mu, sigma = _gradutils(self.endog, self.exog, self.beta, self.alpha)
        self.e1 = self.mu - self.sigma * kappa_0
        sigma2 = self.sigma**2
        self.e2 = (
            sigma2 -
            sigma2 * kappa_1 +
            self.mu**2 -
            2*self.mu*self.sigma*kappa_0
        )
        

    def em(self, start_params, maxiter, use_hessian=False, step_size=1e-4, tol=1e-4):
        # Starting values
        loss = self.nloglikeobs(start_params).mean()
        self.update_beta(start_params[:11])
        self.update_alpha(start_params[11:])
        X = self.exog
        W = self.exog
        penalty_alpha = self.penalty * np.eye(11)
        WtW_plus_penalty = W.T @ W + penalty_alpha
        penalty_beta = penalty_alpha.copy()
        penalty_beta[0] = 0.
        converged = False
        
        for i in range(maxiter):
#             print(f"Iteration {i} loss: {loss}")
            loss_last = loss.copy()
            self.update_expectations()
                                    
            # Calculate beta
            X_sqrt_w = _vec_matrix_multiply(1./self.sigma, X)
            XtSiX = X_sqrt_w.T @ X_sqrt_w
            XtSiX += penalty_beta
            XtSie1 = X.T @ (self.sigma**-2 * self.e1)
            beta = np.linalg.solve(XtSiX, XtSie1)
            
            # Calculate alpha
            c = self.e2 - 2*self.e1*self.mu + self.mu**2 # NOTE: This is using the updated mu
            if use_hessian:
                grad, hessian = _em_gradutils(W, self.sigma, c, self.alpha, return_hessian=True)
                grad -= self.alpha
                hessian -= penalty_alpha
                alpha = self.alpha - np.linalg.solve(hessian, grad)
            else:
                # Backtracking line search
                grad, _ = _em_gradutils(W, self.sigma, c, self.alpha)
                d = -grad # Descent direction
                prop_increase = 0.5 # Called alpha in notes
                step_multiplier = 0.5 # Called beta in notes
                curr_step_size = step_size # Called eta in notes
                f_start = self.nloglikeobs(np.concatenate([self.beta, self.alpha])).sum()
                while True:
                    alpha = self.alpha - curr_step_size * d
                    f_stop = self.nloglikeobs(np.concatenate([self.beta, alpha])).sum()
                    required_change = prop_increase * curr_step_size * (grad @ d)
                    if f_stop - f_start <= required_change:
                        break
                    curr_step_size *= step_multiplier
            
            # Update alpha, beta simultaneously
            self.update_alpha(alpha)
            self.update_beta(beta)

            # Check convergence
            params = np.concatenate([self.beta, self.alpha])
            loss = self.nloglikeobs(params).mean()
            obj = loss_last - loss # Want this to be positive
            if abs(obj) < tol: # Not enforcing any sort of sign constraint for now
                converged = True
                break
        else:
            raise RuntimeError("Hit maxiter and failed to converge")
            
        self.params = np.concatenate([self.beta, self.alpha])
        self.iters = i
        self.loss = loss
        self.loss_last = loss_last
        self.obj = obj
        self.converged = converged

In [6]:
names = list(data.x_df)
names_alpha = [s + "_alpha" for s in names]

In [7]:
mod = MyLatentNormalEM(data.y, data.x_df, penalty=1., extra_params_names=names_alpha)

In [8]:
# Second-order
start_params = np.zeros(22)
start_params[:11] = gpois_res.params[:11] # Warm start with estimates of betas
start_params[11] = -1. # It's really sensitive to this starting value
mod_res = mod.fit(start_params=start_params, maxiter=1000, use_hessian=True, tol=1e-6)
mod_res.wall_time

0.012996196746826172

In [9]:
# First-order
start_params = np.zeros(22)
start_params[:11] = gpois_res.params[:11] # Warm start with estimates of betas
start_params[11] = -1. # It's really sensitive to this starting value
mod_res = mod.fit(start_params=start_params, maxiter=1000, use_hessian=False, step_size=1e-3, tol=1e-6)
mod_res.wall_time

0.35121583938598633

In [10]:
print(mod_res.beta)
print(f"True Beta: {data.beta}")
print(mod_res.alpha)
print(f"True Alpha: {data.alpha}")

[ 5.0362054   0.39143688 -0.15850647  0.27498884  0.28769868  0.38168434
  0.43667161  0.24492697  0.04618936  0.17909704 -0.07685382]
True Beta: [ 5.    0.4  -0.17  0.33  0.36  0.4   0.39  0.26  0.07  0.2  -0.1 ]
[-1.05058743 -0.19967601 -0.05849583  0.25145074  0.42198571  0.45896794
  0.54649341  0.24284324  0.09970537  0.1922257  -0.09557015]
True Alpha: [-1.   -0.2  -0.03  0.33  0.36  0.4   0.39  0.26  0.07  0.16 -0.05]
